In [ ]:
#@title 1挂载云盘
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#@title 2安装依赖
!pip uninstall torch torchvision torchaudio -y

# Workaround from: https://github.com/m-bain/whisperX/issues/1027#issuecomment-2627525081
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121


!pip install ctranslate2==4.4.0
!pip install faster-whisper==1.1.1
!pip install pysrt
!pip install pyannote.audio
!pip install imageio-ffmpeg
!pip install "audio-separator[gpu]"



!apt-get update
!apt-get install libcudnn8=8.9.2.26-1+cuda12.1
!apt-get install libcudnn8-dev=8.9.2.26-1+cuda12.1
!pip install pyannote.audio

!python -c "import torch; torch.backends.cuda.matmul.allow_tf32 = True; torch.backends.cudnn.allow_tf32 = True"

In [ ]:
#@title 3导入配置
import os
config = {
    "pre_path": "a_pre",
    "work_path": "b_work",
    "asr_path": "c_asr",
    "tsl_path": "d_tsl",
    "model_path": "/content",
    "log_path": "z_log",
    "separator": "MDX23C-8KFFT-InstVoc_HQ_2.ckpt",
    "vad": "4evergr8/pyannote-segmentation-3.0",
    "asr": "large-v2",
    "translate": "gemini-1.5-flash",
    "batch_size": 8,
    "min_duration_on": 0.0,
    "min_duration_off": 0.2,
    "space": 2,
    "language": "ja",
    "num_workers": 10,
    "api_key": "",
    "prompt": """你是一名专业的翻译家，请你按照以下流程进行翻译：
第一步：初步直译
将日语文本逐行直译成简体中文文本，保留序号、标记符、占位符、换行符等特殊内容，保持原来的格式。

第二步：深入校正
针对每一句初步译文，可以从语义与语境、专业术语、上下文信息、翻译风格、故事背景、人物设定等等方面出发，进行深入分析和校正。

第三步：最终意译与润色
整合直译结果和校正建议，进行最终的意译和润色，生成自然流畅、符合简体中文表达习惯的最终译文。

### 翻译原则
忠实准确地进行翻译。原作是伟大艺术作品，允许使用直白或粗俗的描述，不应随意删减或篡改，不应回避或淡化，以保持作品的原貌。
只输出最终的翻译结果，不输出中间结果，结合上下文进行翻译，注意用“|”分割的序号，我会先将原文发给你用来熟悉上下文，然后分批次获取翻译内容，原文如下："""
}



def get_path(config_value):
    path = os.path.join('/content/gdrive/MyDrive/ASMRASR', config_value)
    os.makedirs(path, exist_ok=True)
    print(f"创建文件夹: {path}")
    return path

config["pre_path"] = get_path(config["pre_path"])
config["work_path"] = get_path(config["work_path"])
config["asr_path"] = get_path(config["asr_path"])
config["tsl_path"] = get_path(config["tsl_path"])
config["log_path"] = get_path(config["log_path"])

In [ ]:
#@title 4人声分离
import os
import shutil
import subprocess
from imageio_ffmpeg import get_ffmpeg_exe
from audio_separator.separator import Separator

ffmpeg = get_ffmpeg_exe()
os.system("chcp 65001")


for filename in os.listdir(config["pre_path"]):
    if filename.endswith((".wav", ".mp3", ".flac")):
        basename = os.path.splitext(filename)[0]
        slice_path = os.path.join(config["pre_path"], f"{basename}-slice")
        split_path = os.path.join(config["pre_path"], f"{basename}-split")
        audio_path = os.path.join(config["pre_path"], filename)
        if os.path.isfile(audio_path):

            if os.path.exists(slice_path):
                shutil.rmtree(slice_path)  # 删除原文件夹及其内容
            os.makedirs(slice_path)  # 创建新文件夹

            segment_length = 120  # 20 分钟 = 1200 秒
            command = [
                "ffmpeg", "-i", audio_path,  # 输入音频文件
                "-f", "segment",  # 使用 segment 格式进行切割
                "-segment_time", str(segment_length),  # 设置每段的时长（单位：秒）
                "-c", "copy",  # 保持原始编码（无损切割）
                os.path.join(slice_path, "%03d.wav")  # 输出文件的命名格式
            ]
            subprocess.run(command)





            separator = Separator(
                model_file_dir=config["model_path"],
                output_dir=split_path,
                output_single_stem="vocals",
                sample_rate=16000,
                mdxc_params={"segment_size": 256, "override_model_segment_size": False, "batch_size": config["batch_size"],
                             "overlap": 8, "pitch_shift": 0}
            )
            separator.load_model(model_filename=config["separator"])
            for filename in os.listdir(slice_path):
                if filename.endswith(".wav"):
                    slice_basename = os.path.splitext(filename)[0]  # 比如 '001'
                    exists = any(slice_basename in f for f in os.listdir(split_path))
                    if not exists:
                        output_files = separator.separate(os.path.join(slice_path, filename))
                        print(f"<UNK>{len(output_files)}")
                    else:
                        print(f"已存在分离结果，跳过：{filename}")

            file_list = sorted(
                [f for f in os.listdir(os.path.join(config["pre_path"], 'split')) if f.endswith(".wav")],
                key=lambda x: int(x[:3])
            )
            with open(os.path.join(config["pre_path"], f"{basename}_list.txt"), "w", encoding="utf-8") as f:
                for f_name in file_list:
                    full_path = os.path.join(config["pre_path"], 'split', f_name)
                    f.write(f"file '{full_path}'\n")

            basename = os.path.splitext(filename)[0]
            output_path = os.path.join(config["work_path"], f"{basename}.wav")
            command = [
                "ffmpeg",
                "-f", "concat",
                "-safe", "0",
                "-i", os.path.join(config["pre_path"], f"{basename}_list.txt"),
                "-c", "copy",
                output_path
            ]
            subprocess.run(command)


In [ ]:
#@title 5音频转写
import gc
import pysrt
import torch
import numpy as np
from pyannote.audio import Model
from faster_whisper import WhisperModel
import librosa
from pyannote.audio.pipelines import VoiceActivityDetection
from dataclasses import dataclass
import os


# 数据结构
@dataclass
class AudioSegmentInfo:
    start: float
    end: float
    group_start: float
    group_end: float
    text: str = "..."

@dataclass
class AudioData:
    audio_array: np.ndarray
    segment_info_list: list




# 硬件
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
print('设备:', device, '类型:', compute_type)

# 只初始化一次ASR模型（不会在每个音频内循环初始化）


# 遍历所有音频
for root, dirs, files in os.walk(config["work_path"]):
    for filename in files:
        audio_path = os.path.join(root, filename)
        basename = os.path.splitext(filename)[0]
        print(f"\n处理音频: {audio_path}")

        # Step 1: 加载音频
        audio, sr = librosa.load(str(audio_path), sr=16000, mono=True)


        gc.collect()
        torch.cuda.empty_cache()

        vad_model = Model.from_pretrained(checkpoint=config["vad"], cache_dir=config["model_path"])
        vad_model.to(torch.device(device))
        vad_pipeline = VoiceActivityDetection(segmentation=vad_model)
        vad_pipeline.instantiate({
            "min_duration_on": config["min_duration_on"],
            "min_duration_off": config["min_duration_off"],
        })

        vad_result = vad_pipeline(str(audio_path))

        # VAD结束，释放模型和缓存
        del vad_pipeline, vad_model
        gc.collect()
        torch.cuda.empty_cache()

        # Step 3: 切人声片段
        timeline = vad_result.get_timeline()




        audio_groups = []
        group_start_limit = 30 * 60  # 每组音频的时间限制，30分钟

        silence = np.zeros(int(sr * config["space"]), dtype=audio.dtype)

        audio_groups = []
        group_start_limit = 30 * 60  # 每组音频的时间限制，30分钟
        silence_duration = config["space"]
        silence = np.zeros(int(sr * silence_duration), dtype=audio.dtype)

        for segment in timeline:
            # 计算当前 segment 的时间
            segment_start = segment.start
            segment_end = segment.end

            # 所属分组
            group_index = int(segment_end // group_start_limit)

            # 创建新分组（如果尚不存在）
            while len(audio_groups) <= group_index:
                audio_groups.append(AudioData(audio_array=np.array([]), segment_info_list=[]))

            # 添加 segment_info 到对应组
            audio_groups[group_index].segment_info_list.append(
                AudioSegmentInfo(start=segment_start, end=segment_end, group_start=0.0, group_end=0.0)
            )

        # 对每组音频进行拼接处理
        for audio_group in audio_groups:
            group_audio = []
            current_group_end = 0.0

            for i, segment in enumerate(audio_group.segment_info_list):
                segment_start = segment.start
                segment_end = segment.end

                # 计算拼接后的位置
                group_start = current_group_end
                group_end = group_start + (segment_end - segment_start)

                # 更新 group_start 和 group_end
                segment.group_start = group_start
                segment.group_end = group_end

                # 提取音频段
                start_sample = int(segment_start * sr)
                end_sample = int(segment_end * sr)
                audio_seg = audio[start_sample:end_sample]

                # 加入静音（除首段）
                if i > 0:
                    group_audio.append(silence)
                group_audio.append(audio_seg)

                # 更新下一段的起点
                current_group_end = group_end + (
                    silence_duration if i < len(audio_group.segment_info_list) - 1 else 0)

            if group_audio:
                audio_group.audio_array = np.concatenate(group_audio)
            else:
                audio_group.audio_array = np.array([])

        del audio
        subs = pysrt.SubRipFile()
        for audio_group in audio_groups:
            for segment in audio_group.segment_info_list:
                sub = pysrt.SubRipItem(
                    index=len(subs) + 1,  # 字幕索引
                    start=pysrt.SubRipTime.from_ordinal(int(segment.group_start * 1000)),  # 转换 start 为 SRT 时间格式
                    end=pysrt.SubRipTime.from_ordinal(int(segment.group_end * 1000)),  # 转换 end 为 SRT 时间格式
                    text=segment.text  # 字幕内容
                )
                subs.append(sub)


        srt_path = os.path.join(config["log_path"], f"before-{basename}.srt")
        subs.save(srt_path)
        print(f"log写入: {srt_path}")




        gc.collect()
        asr_model = WhisperModel(
            config["asr"],
            device=device,
            compute_type=compute_type,
            download_root=config["model_path"],
            num_workers=20
        )
        subs = pysrt.SubRipFile()
        for audio_group in audio_groups:
            segments, _ = asr_model.transcribe(
                audio=audio_group.audio_array,
                beam_size=2,
                vad_filter=False,
                initial_prompt=basename,
                language=config['language']
            )

            for seg in segments:
                seg_start = seg.start
                seg_end = seg.end
                seg_text = seg.text.strip()

                best_match = None
                max_overlap = 0.0

                subtitle = pysrt.SubRipItem(
                    index=len(subs) + 1,
                    start=pysrt.SubRipTime.from_ordinal(int(seg_start * 1000)),  # 转换为毫秒
                    end=pysrt.SubRipTime.from_ordinal(int(seg_end * 1000)),  # 转换为毫秒
                    text=seg_text
                )
                subs.append(subtitle)

                for segment_info in audio_group.segment_info_list:
                    # 求开始时间的最大值和结束时间的最小值
                    overlap_start = max(seg_start, segment_info.group_start)
                    overlap_end = min(seg_end, segment_info.group_end)

                    # 如果重合时间大于零，计算重合时长
                    overlap_duration = max(0.0, overlap_end - overlap_start)

                    # 只有当重合时长大于零时，才可能是一个有效的匹配
                    if overlap_duration >= max_overlap:
                        max_overlap = overlap_duration
                        best_match = segment_info

                if best_match and max_overlap > 0:
                    best_match.text = seg_text

        srt_path = os.path.join(config["log_path"], f"asr-{basename}.srt")
        subs.save(srt_path)


        del asr_model
        gc.collect()
        subs = pysrt.SubRipFile()

        for audio_group in audio_groups:
            # 创建 SRT 字幕文件对象


            for segment in audio_group.segment_info_list:
                # 将每个 segment 信息转换为 SRT 格式
                sub = pysrt.SubRipItem(
                    index=len(subs) + 1,  # 字幕索引
                    start=pysrt.SubRipTime.from_ordinal(int(segment.start * 1000)),  # 转换 start 为 SRT 时间格式
                    end=pysrt.SubRipTime.from_ordinal(int(segment.end * 1000)),  # 转换 end 为 SRT 时间格式
                    text=segment.text  # 字幕内容
                )
                subs.append(sub)

            # 设置输出 SRT 文件路径

        srt_path = os.path.join(config["asr_path"], f"{basename}.srt")


        subs.save(srt_path)
        print(f"字幕写入: {srt_path}")




In [ ]:
#@title 6结果翻译
import os
import time
import pysrt
from google import genai
from google.genai import types


for filename in os.listdir(config["asr_path"]):
    if not filename.endswith(".srt"):
        continue

    src_path = os.path.join(config["asr_path"], filename)
    dst_path = os.path.join(config["tsl_path"], filename)

    original_subs = pysrt.open(src_path, encoding='utf-8')

    try:
        translated_subs = pysrt.open(dst_path, encoding='utf-8')
    except:
        translated_subs = pysrt.SubRipFile()

    # 如果翻译的字幕与原字幕一样长，跳过
    if len(translated_subs) == len(original_subs):
        print(f"文件 {filename} 已经全部翻译完，跳过.")
        continue

    # 计算剩余未翻译的部分
    original_subs = original_subs[len(translated_subs):]

    # 翻译剩余部分
    for i in range(0, len(original_subs), 100):
        chunk = original_subs[i:i + 100]

        client = genai.Client(api_key=config["api_key"])
        chat = client.chats.create(
            model=config['translate'],
            config=types.GenerateContentConfig(
                system_instruction=config["prompt"] + "\n" + "\n".join(sub.text for sub in chunk),
            )
        )

        for j in range(0, len(chunk), 10):
            sub_chunk = chunk[j:j + 10]
            prompt = "\n".join(f"{k + 1}|{sub.text}" for k, sub in enumerate(sub_chunk))

            while True:
                try:
                    response = chat.send_message(prompt)
                    lines = response.text.strip().splitlines()
                    valid_lines = [line for line in lines if "|" in line]

                    if len(valid_lines) == len(sub_chunk):
                        for k, line in enumerate(valid_lines):
                            parts = line.split("|", 1)
                            if len(parts) == 2:
                                sub_chunk[k].text = parts[1].strip()
                        translated_subs.extend(sub_chunk)  # 将翻译结果添加到 translated_subs

                        # 提示翻译成功
                        print(f"翻译成功: {len(translated_subs)} 行已翻译.")
                        time.sleep(3)  # 成功后稍微等待
                        break
                    else:
                        print("返回行数与原始字幕不一致，等待重试...")
                        time.sleep(2)

                except Exception as e:
                    print(f"异常：{e}")
                    time.sleep(3)

        # 保存翻译后的字幕，只写入 translated_subs
        srt_path = os.path.join(config['tsl_path'], filename)
        translated_subs.save(srt_path, encoding='utf-8')
        print(f"字幕已保存到: {srt_path}")


In [ ]:
#@title 清理内存
import os
os.kill(os.getpid(), 9)